<a href="https://colab.research.google.com/github/justdenz/mco2-technical-report/blob/main/Data_Project_Adding_category_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook adds the commit types category of all commit messages from the [Github Dataset](https://www.kaggle.com/dhruvildave/github-commit-messages-dataset)





In [ ]:
!pip install gdown

In [ ]:
!gdown https://drive.google.com/uc?id=1YuyNK-idGHj1-ltmGkRtok7ml7pP6c6E
!unzip github.zip

Downloading...
From: https://drive.google.com/uc?id=1YuyNK-idGHj1-ltmGkRtok7ml7pP6c6E
To: /content/github.zip
946MB [00:07, 119MB/s]
Archive:  github.zip
  inflating: full.csv                
  inflating: oneline.csv             


In [ ]:
import os
import math
import pickle
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

from datetime import datetime as dt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

import statsmodels.api as sm
import statsmodels.graphics as smg
import statsmodels.stats as sm_stats
import statsmodels.tsa.api as tsa

from scipy import stats


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('full.csv', parse_dates=True, infer_datetime_format=True)

In [ ]:
import re
from typing import List

Adding the categories features an intricute regex matching using keywords. These functions attempt to match the category as either:

*   Fix
*   Core Bug
*   Refactor
*   Security
*   Others

The "other" categories included addition of new features, or general improvent.

This process is mainly adapted from this [example](https://www.kaggle.com/amarabuco/commits-eda).



Language Util - This is the base regex reading methods.


In [ ]:
regex_list : List[str]

SCHEMA_NAME = 'general'
file_scheme = '([a-zA-Z0-9_\*\.])+\.[a-zA-Z]{1,4}'

REGULAR_SUFFIX = '(?:s|ed|ing)?'
VERB_E_SUFFIX = '(?:e|es|ed|ing)'

NEAR_ENOUGH = '[\S\s]{0,40}'

#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|-|\|)" # Adding - should be tuned
#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|\|)"
#term_seperator = "[^abcdefghijklmnopqrstuvwxyz]"
term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|_|\|)"
#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|/|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|_|\|)" # no ",'
#term_seperator = "(\s|\.|\?|\!|\[|\]|\(|\)|\:|^|$|\,|\'|\"|#|\$|\%|&|\*|\+|=|`|;|<|>|@|~|{|}|_|\|)" # without /

# Negation
negation_terms = ["aren't",
 'arent',
 "can't",
 'cannot',
 'cant',
 'could not',
 "couldn't",
 'couldnt',
 "didn't",
 'didnt',
 "doesn't",
 'doesnt',
 "don't",
 'dont',
 "hasn't",
 "haven't",
 "isn't",
 'isnt',
 'lack',
 "n't",
 'never',
 'no',
 'nobody',
 'none',
 'not',
 'nothing',
 "shouldn't",
 'shouldnt',
 "weren't",
 'werent',
 'without',
 "won't",
 'wont',
 "wouldn't",
 'wouldnt']

# TODO - consider adding if, maybe
modals = [#'if', 'maybe',
          'can', 'could', 'ha(?:ve|s|d)', 'may', 'might', 'must', 'need', 'ought', 'shall'
    , 'should', 'will', 'would']



# TODO - check https://arxiv.org/pdf/2001.09148.pdf for more

security_terms = [
 'advisory',
 'attack',
 'authenticat(e|ion)',
 'brute force', # consider
 'bug bount(y|ies)',
 'bypass', # consider
 'constant time',
 'crack',
 'credential(s)?',
 'cross-origin',
 'cross site',
 'cve(-d+)?(-d+)?',
 'clickjack',
 'cyber',
 'denial of service',
 '(de)?serializ', # consider
 'directory traversal',
 'dos', # consider
 'exploit',
 'expos(e|ing)',
 'hack',
 'hijack',
 'harden',
 #'infinite loop', # consider
 'injection',
 '(in)?secur(e|ity)',
 'lockout',
 'malicious',
 'malware',
 'nvd' # NVD
 'open redirect',
 'osvdb', # OSVDB
 'overflow', # consider
 'password(s)?',
 'permission(s)?',
 'poison',
 'port scan',
 'privilege',
 # 'proof of concept', # consider
 'rce', # remote code execution
 'redos' # ReDoS
 'remote code execution',
 'return oriented programming',
 'security',
 'session fixation',
 'spoof',
 'threat',
 'timing', # consider
 'traversal',
 'unauthori[z|s]ed',
 'vulnerabilit(?:y|ies)',
 'x(?: |-)frame(?: |-)options',
 'xss',
 'xsrf', # XSRF
 'xxe' # XXE
    ]


documentation_entities = [
    'change(?:s)?(?: |-)?(list|log|set|file)',
    'comment(s)?',
    'copy(?: |-)?right(?:s)?',
    'doc(?:s)?',
    'documentation',
    'explanation(?:s)?',
    'man(?: |-)?page(?:s)?',
    'manual',
    'note(?:s)?',
    'readme(?:.md)?',
    r'[-a-z\d_/\\]*.(md|txt)',
    'translation(?:s)?',
    'java(?: |-)?doc(?:s)?',
    'java(?: |-)?documentation',
    'example(?:s)?',
    'diagram(?:s)?',
    'guide(?:s)?',
    'gitignore',
    'icon(?:s)?',
    'doc(?: |-)?string(?:s)?',
    'tutorials(?:s)?',
    'help',
    'man',
    'doc(?: |-)?string(?:s)?',
    'desc(?:ription)?(?:s)?',
    'copy(?: |-)?right(?:s)?',
    'explanation(?:s)?',
    'release notes',
    'tag(?:s)?', # Git commit tags

]

prefective_entities = documentation_entities +[
    'indentation(?:s)?'
    , 'style'
    , 'todo(s)?'
    , 'typo(s)?'
    , 'verbosity']

software_goals = ['abstraction', 'coherence', 'cohesion', 'complexity', 'correctness', 'coupling', 'dependability'
    , 'duplication', 'efficiency', 'extensibility', 'flexibility' ,'maintainability', 'naming', 'performance', 'portability', 'quality'
    , 'readability', 'reliability', 're(?:-| )?use' ,'re(?:-| )?usability', 'security', 'simplicity', 'testability', 'testable', 're(?:-| )?usable'
    , 'readable', 'portable', 'maintainable', 'flexible', 'efficient', 'encapsulation'
                  ]

software_goals_modification = [
    'better','improv(?:e|es|ed|ing)', 'increas(?:e|es|ed|ing)', 'reduc(?:e|es|ed|ing)', 'worse', 'make', 'more', 'less'
]

software_entities = ['algorithm(?:s)?',
 'class(?:es)?',
 'collection(?:s)?',
 'constant(?:s)?',
 'constructor(?:s)?',
 'field(?:s)?',
 'function(?:s)?',
 'interface(?:s)?',
 'member(?:s)?',
 'method(?:s)?',
 'module(?:s)?',
 'parameter(?:s)?',
 'procedure(?:s)?',
 'routine(?:s)?`',
 'structure(?:s)?',
 'template(?:s)?',
 'type(?:s)?',
 'unit(?:s)?',
]

software_terms = [ 'assertion(?:s)?', 'assignment(?:s)?',  'code',  'conditional(?:s)?',  'control', 'definition(?:s)?'
    , 'delegate', 'delegation'
    , 'design pattern(?:s)?', 'error(?:-| )?code(?:s)?', 'exception(?:s)?',  'flag(?:s)?',  'getter(?:s)?'
    , 'guard clause(?:s)?', 'hierarch(?:y|ies)', 'implementation(?:s)?', 'inheritance', 'inline'
    ,  'internal', 'macro(?:s)?'
    , 'magic number(?:s)?', 'modifier(?:s)?', 'null object(?:s)?', 'object(?:s)?'
    , 'patch(?:es)?',  'pointer(?:s)?', 'polymorphism', 'quer(?:y|ies)',  'reference(?:s)?'
    , 'ref(?:s)?'
    , 'return type', 'setter(?:s)?', 'static',  'sub(?:-| )?class(?:es)?', 'super(?:-| )?class(?:es)?'
    , '(?:sub)?(?:-| )?system(?:s)?'
    , 'uninline'
    #, 'value(?:s)?'
    , 'variable(?:s)?', 'handler', 'plugin'
    #, '(?:in)?validation'
    #, 'input', 'output'
    , 'contravariant', 'covariant'
                  # , 'link(?:s)?'
    ,
                  'action(?:s)?'
                  # , 'event(?:s)?'
    , 'queue(?:s)?', 'stack(?:s)?'
    #, 'change(?:\s)?log'
    , 'driver(?:s)?'
    #, 'hook(?:s)?'
    #, 'target(?:s)?'
    , 'storage', 'tool(?:s)?',  'log(?:s)?', 'setting(?:s)?'
    #, '(?:index|indexes|indices)'
    , 'fall(?: |-)back(?:s)?', 'memory', 'param(?:s)?', 'volatile', 'file(?:s)?'
    , 'generic(?:s)?'
    #, 'test(?:s)?'
    , 'initialization(?:s)?', 'public', 'protected', 'private' ,'framework', 'singelton', 'declaration(?:s)?'
    , 'init' , 'destructor(?:s)?', 'instances(?:s)?', 'primitive(?:s)?'
    #, 'middle man'
    #, 'hierarchy'
                  ] + software_entities


# Well, we need them...
unnedded_terms = ['unnecessary', 'unneeded', 'unused', '(?:not|never|no longer) used'
    #, 'old'
    , 'no longer needed', 'redundant', 'useless', 'duplicate(?:d)?', 'deprecated', 'obsolete(?:d)?', 'commented']


static_analyzers = ['lint', 'pylint', 'tslint', 'jlint', 'jslint', 'eslint', 'klint', 'xlint', 'linter']

code_review_fixes = ['(cr|pr)(s)?(-)?(d+)?\sfix(es)?', 'fix(?:ing|es|ed)?\s(cr|pr|code review|code-review|review)']

no_message = ['no message', 'wip', 'work in progress', 'message', 'change(?:-|\s)?set', 'commit']

programming_languges = [i.lower() for i in ['Python', 'JavaScript', 'Java', 'C\+\+', 'PHP', 'TypeScript', 'C',
       'C\#', 'Go', 'Ruby', 'HTML', 'Shell', 'CSS', 'Kotlin', 'Scala',
       'Swift', 'Jupyter Notebook', 'Rust', 'Perl', 'Lua', 'Haskell', 'R',
       'Objective\-C', 'Groovy', 'Vue', 'PowerShell', 'TSQL', 'Dart',
       'Clojure', 'MATLAB', 'Emacs Lisp', 'OCaml', 'Erlang', 'Elixir',
       'CoffeeScript', 'TeX', 'Fortran', 'Assembly', 'Vim script',
       'PLpgSQL', 'Makefile', 'Julia', 'BitBake', 'F\#', 'Common Lisp',
       'Vala', 'Coq', 'Smalltalk', 'Scheme', 'Visual Basic .NET',
       'Puppet', 'HCL', 'Smarty', 'Dockerfile', 'XSLT', 'GLSL', 'Haxe',
       'Cuda', 'Ada', 'SQF', 'Pascal', 'PLSQL', 'Gherkin', 'Jsonnet',
       'Nix', 'Roff', 'Apex', 'QML', 'CMake', 'D', 'Perl 6',
       'Visual Basic', 'Objective\-C\+\+', 'Prolog', 'Mathematica',
       'Batchfile', 'Reason', 'Markdown', 'DM', 'Elm', 'FreeMarker',
       'ABAP', 'M4', 'SystemVerilog', 'AutoHotkey', 'Verilog', 'IDL',
       'Tcl', 'Rich Text Format', 'SaltStack', 'UnrealScript', 'Zig',
       'WebAssembly', 'RAML', 'F\*', 'Stan', 'ColdFusion', 'Factor',
       'LLVM', 'Pike', 'VBA', 'Isabelle', 'OpenSCAD', 'ASP', 'Arc',
       'Racket', 'LookML', 'SMT', 'q', 'Xojo', 'ZenScript', 'Ceylon',
       'Agda', 'Limbo', 'SuperCollider', 'Pawn', 'xBase', 'JSON', 'Nim',
       'M', 'XC', 'SourcePawn', 'GDScript', 'LilyPond', 'SQLPL',
       'PostScript', "Ren'Py", 'Gnuplot', 'OpenEdge ABL',
       'Common Workflow Language', 'Xtend', 'Mercury', 'Genshi',
       'Open Policy Agent', 'RobotFramework']]


def build_sepereted_term(term_list : List, just_before =False):
    if just_before:
        sep = "%s(%s)" % (term_seperator, "|".join(term_list))
    else:
        sep = "%s(%s)%s" % (term_seperator, "|".join(term_list), term_seperator)
    return sep


def build_non_positive_linguistic(positive_re
                                  , neg=negation_terms):

    non_actionable_context = ['for(?:get|gets|got|getting)'
        , 'allow(s|ed|ing)?']


    return '(?:%s)' % "|".join([
        ('(?:%s)' + NEAR_ENOUGH + '(?:%s)') % (build_sepereted_term(modals, just_before=True)
                                      ,  positive_re)
        , ('(?:%s)' + NEAR_ENOUGH + '(?:%s)') % (build_sepereted_term(neg, just_before=True)
                                        ,  positive_re)
        , ('(?:%s)' + NEAR_ENOUGH + '(?:%s)') % (build_sepereted_term(non_actionable_context, just_before=True)
                                        ,  positive_re)
        # TODO - take care of documentation entities spereatly
        #, '(?:%s)[\s\S]{0,10}(?:%s)' % (build_sepereted_term(documentation_entities, just_before=True)
        #                                ,positive_re)
    ])


def match(commit_text, regex):
    text = commit_text.lower()

    return len(re.findall(regex, text))


def regex_to_big_query(reg_exp
                       , text_field='message'):
    # TODO - check
    # Take care of encoding
    reg_exp = reg_exp.replace("\\", "\\\\").replace("'", "\\'")
    #reg_exp = reg_exp.replace("\\\\", "\\")
    # No need for grouping
    reg_exp = reg_exp.replace("(?:", "(")
    str = "(" + "LENGTH(REGEXP_REPLACE(lower(" + text_field + ")," + "'%s', '@'))" % reg_exp + "-" \
          + "LENGTH(REGEXP_REPLACE(lower(" + text_field + ")," + "'%s', ''))" % reg_exp + ")"

    return str


def generate_bq_function(func_name
                         , code_generator
                         , commit: str ='XXX'):
    print("# Run in Standard sql ")
    print("CREATE OR REPLACE FUNCTION ")
    print(func_name)
    print(" (message string) ")
    print(" RETURNS int64 ")
    print("AS (")
    print("# Model language based on commit: {commit} ".format(commit=commit))
    code_generator()
    print(" ) ")
    print(" ; ")


def normalize(string):
    string = re.sub(r"\s+", " ", string.strip())
    while "  " in string:
        string = string.replace("  ", " ")
    return string

def print_logic_to_bq(regex_func
                      , concept):
    print("# " + concept)
    print( regex_to_big_query(regex_func()))
    print("# " + concept + " - end")

Conventional Commit - This filtes the format for conventional commits. Conventional commits is a convention that enforces strict labelling and descriptive message for commits so as they are easily identifyable.

In [ ]:
cc_adaptive_terms = ['feat' # Feature
                        , 'build'
                        , 'chore'
                        , 'ci' # continuous integration
                        , 'test'
                        , 'perf' # performance
                     ]
cc_corrective_terms = ['fix']
cc_perfective_terms = ['docs', 'style']
cc_refactor_terms = ['refactor']

cc_actions = cc_adaptive_terms + cc_corrective_terms + cc_perfective_terms + cc_perfective_terms

cc_etc = ['breaking\s+change(!)?:']


def cc_title(astions):

    return '^(' + "|".join(astions) +")(\(.*\))?(!)?:"


# Adaptive
def build_cc_adaptive_regex():

    return cc_title(cc_adaptive_terms)


def is_cc_adaptive(commit_text):

    return len(re.findall(build_cc_adaptive_regex(), commit_text)) > 0


# Corrective
def build_cc_corrective_regex():

    return cc_title(cc_corrective_terms)


def is_cc_corrective(commit_text):

    return len(re.findall(build_cc_corrective_regex(), commit_text)) > 0

# Refactor
def build_cc_refactor_regex():

    return cc_title(cc_refactor_terms)


def is_cc_refactor(commit_text):

    return len(re.findall(build_cc_refactor_regex(), commit_text)) > 0

# Just Perfective
def build_cc_just_perfective_regex():

    return cc_title(cc_perfective_terms)


def is_cc_just_perfective(commit_text):

    return len(re.findall(build_cc_just_perfective_regex(), commit_text)) > 0

# Perfective
def build_cc_perfective_regex():

    return "(" + "|".join([build_cc_refactor_regex()
                              , build_cc_just_perfective_regex()]) + ")"


def is_cc_perfective(commit_text):

    return len(re.findall(build_cc_perfective_regex(), commit_text)) > 0

# CC message
def build_cc_message_regex():

    return "(" + "|".join([cc_title(cc_actions)] + cc_etc) + ")"



def is_cc_message(commit_text):

    return len(re.findall(build_cc_message_regex(), commit_text)) > 0




def print_cc_functions_for_bq(commit: str = 'XXX'):

    concepts = {'cc_adaptive' : build_cc_adaptive_regex
                , 'cc_corrective' : build_cc_corrective_regex
                , 'cc_refactor' : build_cc_refactor_regex
                , 'cc_just_perfective' : build_cc_just_perfective_regex
                , 'cc_perfective' : build_cc_perfective_regex
                , 'cc_message' : build_cc_message_regex
                }

    for i in concepts.keys():
        print()
        print_func = lambda : print_logic_to_bq(regex_func=concepts[i]
                                                , concept=i)
        generate_bq_function('{schema}.bq_{concept}'.format(schema=SCHEMA_NAME
                                                            , concept=i)
                             , print_func
                             , commit=commit)
        print()

**Corrective/Bug**

Keywords

In [ ]:
core_bug_terms = [
             'bug(s|z)?',
             'bug(?:-|\s)?fix(es)?',
             'defect(?:s)?',
             'error(?:s)?',
             'failur(?:ing|e|es|ed)',
             'fault(s)?',
             'fix(ed|es|ing)?',
             'fixing(?:s)?',
             'incorrect(ly)?',
             'mistake(s|n|d|nly)?',
             'problem(?:s)?',
             ]
# Positive
bug_terms = ['actual.*expected',
             '((assignment|assign|=) in if|== instead of =)',
             'expected.*actual'
             '(choose|take|set|use)\\s*(the|a)?\\s*correct', # correct as adjective
             "(not|isn't|doesn't)\\s+work(s|ing)?", # TODO - check with negation
             "doesn't recognize", # TODO Extend
             'double(?:-| )allocat(?:e|ion|ions)',
             'double(?:-| )free(?:s)?',
             "caused a regression", # TODO Extend
             'bad initialization(?:s)?',
             'buffer overflow(?:s)?',
             'fixme(?:s)?',
             'fixes(?:-| )?commit(?::| )?',
             '(break|breaks|broke|broked|breaking|broken)[\s\S]{0,20}(code|system|function|method)',
             '(this|that|it)\s(break|breaks|broke|broked|breaking|broken)',
             'break strict(?:-|\s)aliasing rule(s)?',
             'crash(?:ing|s|ed)?',
             'correct(?:ing|s|ed)?\\s*(a|the|some|few|this)', # make sure that correct serves as a verb
             'correct(ed|ion|ly|s)?',
             '(dangling|hanging) pointer(?:s)?',
             'deadlock(?:s)?',
             '(divid(e|es|ed|ing)|division) by (zero|0)',
             'double(?:-| )free',
             'fail(?:ing|s|ed)?',
             'faulty initialization(?:s)?',
             'fix(?:-| )?in(?:s)?',
             'fix(?:-| )?up(?:s)?',
             'flaw(?:s|ed)?',
             '(float|integer) (under|over)(?:-| )?flow',
             'hot(?:-| )?fix(?:ed|es|ing)?',
             #'hang',
             'heap overflow(?:s)?',
             '(?:im|im-)?proper'
             'memory(?:-| )?leak(?:s)?',
             'missing\s(default value|initialization|switch case)(?:s)?',
             'is\smissing',
             'add(?:ing|s|ed)?\smiss(?:ing|es|ed)?',
             #'must not',
             'npe(?:s)?'
             'null pointer(?:s)?',
             'off(?:-| )by(?:-| )(one|1)',
             'out of bound(?:s)?',
             'over(?:-| )?run(?:s)?',
             'patch(?:ed|ing)',
             #'prevent(?:ing|s|ed)?', # should be tuned
             'race condition(?:s)?',
             'data race(?:s)?',
             'repair(?:ing|s|ed)?',
             'resource leak(?:s)?',
             # TODO - check generalization to leaks works in the other direction to expected (reduces FP, increases FN)
             'leak(?:s)?',
             'revert(?:ing|s|ed)?',
             'segmentation (fault|violation)(?:s)?',
             'resolv(?:ing|e|es|ed)',
             #'solv(?:ing|e|es|ed)',
             'workaround(?:s)?',
             'wrong(nly)?',
             '(type(s)? mis(?:-| )?match|(not|non|none) matching type(s)?)',
             'trouble(?:s)?',
             '(un(?:-| )?|not )initialized variable(s)?',
             'unintended',
             'not intended',
             'unintentionally',
             'not intentionally',
             # 'unexpected.*occurred', # very rare, 90% are bugs anyway
             'vulnerabilit(?:y|ies)'
             ] + core_bug_terms

# Valid_fix_objects
valid_fix_object = prefective_entities + ['#',
                    '(camel|snake|kebab|flat|lower|upper)\\s*case',
                    "code review('s|s)?",
                    'comment(?:s)?',
                    'cosmetic(?:s)?',
                    'cr(s)?(?:-)?',
                    'documentation(?:s)?',
                    #'exception(?: |-)?handling',
                    #'format(s|ing)? fix(ed|es|ing)?',
                    'format(?:ing)?',
                    'help',
                    'remark(s)?',
                    'space(s)?',
                    'style|styling',
                    'typo(s)?',
                    'typing(?: |-)?(error|mistake)(s)?',
                    'warning(s)?',
                    'white(?: |-)?space(s)?']

valid_terms = [
    'break\sout',
    'error(?: |-)?check(ing)?',
    'error(?: |-)?handling',
    'error message(s)?',
    'error report(s|ing)?',
    'fixed(?: |-)?point',
    'fix(?:ed) ticket(?:s)?',
    #'format(ing)?',
    '(?:fix(?:ed|es)?|bug)(?: )?(?: |-|=|:)(?: )?[a-z]{0,3}(?:-)?\d+' + term_seperator,
    '(if|would)[\s\S]{0,40}go wrong',
    'line(?:s)? break(?:s)?',
    'typo(s)?\sfix(es)?',
    'fix(ed|es|ing)?' + build_sepereted_term(software_entities) + 'name(s)?',
    build_sepereted_term(static_analyzers) + 'fix(es|ed)?',
    'fix(es|ed)?' + build_sepereted_term(static_analyzers) ,
    '^### Bug Fix', # tends to be a title, later stating if the commit is a bug fix
    'edit the jira link to the correct issue', # Another occurring title
    'page(?:s)? break(?:s)?',


] + code_review_fixes



fixing_verbs = ['correct(?:ing|s|ed)'
                    , 'fix(ed|s|es|ing)?'
                    , 'repair(?:ing|s|ed)?'
                    ,  'revert(?:ing|s|ed)?'
                    , 'resolv(?:ing|e|es|ed)'
                    , 'revok(?:ing|e|es|ed)'
                    , 'und(?:oing|id)'
                ]
MERGE_PREFIX = '(merge (branch|pull request).{0,25}|merge (branch|pull request).{0,25}(from|into).{0,25})'
END_OF_LINE = r'(\r\n|\r|\n|$)'
corrective_header_entities = fixing_verbs + [
    'miss(?:ing|es|ed)?', 'should', 'must', '(have|has) to', 'avoid', 'prevent', 'break(s|ed|ing)?', 'broken'
    , 'remov(?:ing|e|es|ed) change(?:s)?', 'unable', 'proper(?:ly)?'
    , MERGE_PREFIX + "(%s)" % "|".join(core_bug_terms) + '.{0,250}' + END_OF_LINE
    #, "(does not|doesn't) need" , "cannot", "can not"
 ] #+ [ "do not" ,"don't", "dont"]

Regex Functions

In [ ]:
def build_sepereted_term(term_list : List, just_before =False):
    if just_before:
        sep = "%s(%s)" % (term_seperator, "|".join(term_list))
    else:
        sep = "%s(%s)%s" % (term_seperator, "|".join(term_list), term_seperator)
    return sep

In [ ]:
def build_valid_find_regex():
    fix_re = "(" + "|".join(fixing_verbs + [MERGE_PREFIX]) + ")"
    prefix = term_seperator + fix_re + '[\s\S]{1,40}' + "(" + "|".join(valid_fix_object) + ")" + term_seperator

    suffix = term_seperator + "(" + "|".join \
        (valid_fix_object) + ")" + term_seperator + '[\s\S]{0,40}' + term_seperator + fix_re + term_seperator

    # TODO - check seperation
    #sepertion = '(?:%s|%s[\s\S]{0,40}%s)' % (term_seperator, term_seperator, term_seperator)
    #suffix = "(" + "|".join \
    #    (valid_fix_object) + ")" + sepertion + fix_re + term_seperator

    #other_valid_re = "(%s)" % "|".join(valid_terms)
    other_valid_re = build_sepereted_term(valid_terms)
    return "((%s)|(%s)|(%s))" % (prefix, suffix, other_valid_re)

In [ ]:
def build_bug_fix_regex(use_conventional_commits=True):
    header_regex =  '(?:^|^[\s\S]{0,25}%s)(?:%s)%s' % (term_seperator
                                                       , "|".join(corrective_header_entities)
                                                       , term_seperator)
   # strict_header = "^(?:%s)%s"  % ( "|".join([ "do not" ,"don't"])
   #                                                    , term_seperator)

    bug_fix_re = build_sepereted_term(bug_terms)


    if use_conventional_commits:
        agg_re = "((%s)|(%s)|(%s))" % (bug_fix_re, header_regex, build_cc_corrective_regex())
    else:
        agg_re = "((%s)|(%s))" % (bug_fix_re, header_regex)

    return agg_re

In [ ]:
def build_negeted_bug_fix_regex():
    bug_fix_re = build_bug_fix_regex(use_conventional_commits=False)
    negation_re = build_sepereted_term(negation_terms)


    return "%s[\s\S]{0,20}%s" % (negation_re, bug_fix_re)

In [ ]:
def build_core_bug_regex():

    return '(%s)' % build_sepereted_term(core_bug_terms)

**Security**

Keywords

In [ ]:
positive_terms = [
 'advisory',
 'attack(?:s)?',
 'auth',
 'authenticat(e|ion)',
 'brute force', # consider
 'bug bount(y|ies)',
 'bypass(?:es|ed|ing)?', # consider
 'certificate(?:s)?',
 #'constant time', # too general
 'crack',
 'credential(s)?',
 'cross(?: |-)origin',
 'cross(?: |-)site',
 'cve(-d+)?(-d+)?',
 'clickjack',
 'cyber',
 'denial of service',
 '(de)?serializ', # consider
 'directory traversal',
 'dos', # consider
 'exploit',
 #'expos(e|ing)',
 # 'hack', # A bit general, consider
 'hijack',
 'harden',
 #'infinite loop', # consider
 'injection',
 '(in)?secur(e|ity)',
 'lockout',
 'malicious',
 'malware(?:s)?', #plural of malware is malware yet not all are aware
 'nvd' # NVD
 'open redirect',
 'osvdb', # OSVDB
 'overflow', # consider
 'password(?:s)?',
 'permission(?:s)?',
 'poison(?:s|es|ed|ing)?',
 'port scan',
 'privilege(?:s)?',
 # 'proof of concept', # consider
 'rce', # remote code execution
 'redos' # ReDoS
 'remote code execution',
 'return oriented programming',
 '(?:safe|safety|unsafe|safer)',
 'security',
 'session fixation',
 'spoof(?:s|es|ed|ing)?',
 'threat(?:s|ed|ing)?',
 'timing', # consider
 'token(?:s)?',
 #'traversal',
 'unauthori[z|s]ed',
 'vulnerabilit(?:y|ies)',
 'x(?: |-)frame(?: |-)option(?:s)?',
 'xss',
 'xsrf', # XSRF
 'xxe' # XXE
    ]

excluded_terms = ['_____PLACEHOLDER_____'
                  ]

Regex Functions

In [ ]:
def build_positive_regex():

    return build_sepereted_term(positive_terms)



def build_excluded_regex():

    return build_sepereted_term(excluded_terms)

def build_not_positive_regex():

    return build_non_positive_linguistic(build_positive_regex())

**Other**

Keywords

In [ ]:
core_adaptive_terms = [
    'add(?:s|ed|ing)?',
    'creat(?:e|es|ing)',
    'disabl(?:e|es|ed|ing)',
    'implement(?:ed|s|ing)?',
    'import(?:s|ed|ing)?',
    'introduc(?:e|es|ed|ing)',
    'port(?:s|ed|ing)?',
    'provid(?:e|es|ed|ing)',
    'updat(?:e|es|ed|ing)',
    'upgrad(?:e|es|ed|ing)'

]

adaptive_context = [
 '(?:un)?hid(?:e|es|den)',
 'allow(?:s|ed|ing)?',
 'buil(?:t|ds|ing)',
 'calibirat(?:e|es|ed|ing)',
 'configure',
 'deferr(?:ed|s|ing)?',
 'enhanc(?:e|es|ed|ing)',
 'extend(?:s|ed|ing)?',
 'form(?:ed|s|ing)?',
 'report(?:s|ed|ing)?',
 'support(s|ed|ing)?',

# , 'mov(e|es|ed|ing)'
# , 'print(s|ed|ing)?'

] + core_adaptive_terms



adaptive_entities = ['ability', 'configuration', 'conversion', 'debug', 'new', 'possibility', 'support'
    , 'test(s)?', 'tweak(s)?', 'mode', 'option']


adaptive_header_action = "|".join([
    'upgrad(?:e|es|ed|ing)',
    'configur(?:e|es|ed|ing)',
    'chang(?:e|es|ed|ing)',
    '(?:keep|change)\s+(?:the\s+)?default',
    'new',
    # '(?:make(?:s)?|made|making)',
    'merg(?:e|es|ed|ing)',
    'clear(?:s|ed|ing)?',
    #'comment(?:s|ed|ing)?\sout'
    'creat(?:e|es|ed|ing)',
    'cast(?:s|et|ing)?' + NEAR_ENOUGH + '\sas',
    # 'convert(?:s|ed|ing)?',
    # 'check(?:s|ed|ing)?',
    'add(?:s|ed|ing)?',
    # 'buil(?:d|t|ds|ing)',
    'Initial revision',
    '(?:im)?port(?:s|ed|ing)?',
    '(?:un)?hid(?:e|es|den)',
    'updat(?:e|es|ed|ing)',
    'upload(?:s|ed|ing)?',
    'disabl(?:e|es|ed|ing)',
    'delet(?:e|es|ed|ing)',
    'enabl(?:e|es|ed|ing)',
    'quirk(?:s|ed|ing)?',
    'skip(?:s|ed|ing)?',
    'switch(?:s|ed|ing)?',
    'allow(?:s|ed|ing)?',
    'provid(e|es|ed|ing)',

    ###
    # , 'build'
    # , 'mark(?:s|ed|ing)?'
    # , 'us(?:e|es|ed|ing)'
    # , '(?:make|made|making)'
    # , 'creat(?:e|es|ed|ing)'
    # , 'handl(?:e|es|ed|ing)'
    'remov(?:e|es|ed|ing)',
    'refresh(?:s|ed|ing)?',
    #'re(-)?enabl(?:e|es|ed|ing)',

] +no_message
)

adaptive_actions = [  # 'revert(?:s|ed|ing)?',
    #'merg(?:e|es|ed|ing)[\s\S]{1,5}(pull request|pr|branch)',
    'add(?:s|ed|ing)?[\s\S]{1,50}(?:version|v\d|ver\d)',
    #'(cr(s)?(-)?|code\sreview)\sfix(?:s|ed|ing)?',
    '(^|\s)implement(?:ed|s|ing)?\s',
    '(?:make(?:s)?|made|making)[\s\S]{1,50}consitent',
    'updat(?:e|es|ed|ing)[\s\S]{1,25}to[\s\S]{1,25}\d+.\d',
    'updat(?:e|es|ed|ing)\s+(to\s+)?\d+\.\d',
    '(?:add(s|ed|ing)?|delet(?:e|es|ed|ing)|updat(?:e|es|ed|ing))\s+' + file_scheme,
    # '(add(s|ed|ing)?|delet(e|es|ed|ing)|updat(e|es|ed|ing))\s+([A-Z0-9_]*)', # TODO - run without lower
    '(^|^[\s\S]{0,25}%s)(%s)%s' % (term_seperator, adaptive_header_action, term_seperator),
    # '^(?:version|v\d+\.\d|ver\d+\.\d)',
    '^\[(?:IMP|imp)\]',  # TODO - take care of upper/lower case
    'support(?:s|ed|ing)?\sfor\s',
    'show(?:es|ed|ing)?[\s\S]instead',
    'scal(?:e|es|ed|ing)?\s(up|down)'

                   ] + code_review_fixes

In [ ]:
def build_adaptive_action_regex():

    return "(%s)" % ("|".join(
    adaptive_actions))




def build_adaptive_regex(use_conventional_commits=True):

    adaptive_context_re = build_sepereted_term(adaptive_context, just_before=True)


    base_re = "((%s)\s[\s\S]{0,50}(%s)%s)" % (adaptive_context_re
                            ,  "|".join(adaptive_entities + software_terms)
                            , term_seperator)

    if use_conventional_commits:
        agg_re = "((%s)|(%s))" % (base_re
                                  , build_cc_adaptive_regex())
    else:
        agg_re = base_re

    return agg_re



def build_non_adaptive_context():

    non_adaptive_header_action = "|".join([
                                'transla(?:tion|et|eted|ets|ting)'
                                ,  'readme(?:.md)?'
                              ])

    non_adaptive_header ='^[\s\S]{0,50}(%s)' % non_adaptive_header_action

    entities = documentation_entities + ['bug',
                'helper',
                'miss(?:ing|ed)',
                'to(?: |-)?do(?:s)?',
                'warning(?:s)?'
                ]

    adaptive_actions = ['remov(?:e|es|ed|ing)']
    non_adaptive_entities = documentation_entities + software_terms + unnedded_terms + [file_scheme]


    return '(%s)' % "|".join(['(?:%s)\s[\s\S]{0,50}(?:%s)' % (build_sepereted_term(adaptive_context, just_before=True)
                                                            , "|".join(entities))
                     , non_adaptive_header
                     , '(?:%s)\s[\s\S]{0,50}(?:%s)' % (build_sepereted_term(adaptive_actions, just_before=True)
                                                            , "|".join(non_adaptive_entities))
                     ])


def build_non_adaptive_linguistic():

    return build_non_positive_linguistic(build_adaptive_regex(use_conventional_commits=False))

def build_core_adaptive_regex():

    return '(%s)' % build_sepereted_term(core_adaptive_terms)

**Refactor**

Keywords

In [ ]:
# https://arxiv.org/pdf/2002.11049.pdf
refactor_entities = software_terms + ['(helper|utility|auxiliary) function(?:s)?']


# Well, we need them...
unnedded_terms = ['unnecessary', 'unneeded', 'unused', '(?:not|never|no longer) used'
    #, 'old'
    , 'no longer needed', 'redundant', 'useless', 'duplicate(?:d)?', 'deprecated', 'obsolete(?:d)?', 'commented']

core_refactor_terms = [
    'clean(?:ing)?(?:-| )?up(?:s)?',
    'clean(?:ing|s|ed)?',
    'combin(?:e|es|ed|ing)',
    'compos(?:e|es|ed|ing)',
    'de(?:-| )?compos(?:e|es|ed|ing)',
    'deprecat(?:e|es|ed|ing)',
    'encapsulat(?:e|es|ed|ing)',
    'polish(?:ed|es|ing)?',
    're(?:-| )?factor(?:ed|s|ing|ings)?', # TODO - should be here - check why slightly decreases performance
    're(?:-|)?organiz(?:e|es|ed|ing)',
    're(?:-|)?structur(?:e|es|ed|ing)',
    'rebuil(?:d|ds|ding|t)',
    'tid(?:y|ying|ied)'
]


modification_activity = [
'(?:get|got|getting) rid',
 '(?:make|makes|made|making)',
 'convert(?:ed|s|ing)?',
 'dead',
 'drop(?:ed|s|ing)?',
 'duplicat(?:e|es|ed|ing)',
 'extract(?:ed|s|ing)?',
 'hide(?:e|es|ed|ing)',
 'improv(?:e|es|ed|ing)',    # Goals modification only?
 'increas(?:e|es|ed|ing)',
 'mov(?:e|es|ed|ing)',
 'parameteriz(?:e|es|ed|ing)',
 'redundant',
 'replac(?:e|es|ed|ing)',
 'separat(?:e|e s|ed|ing)',
 'short(:?en|er|ing|s)?',
 'split(?:s|ing)?',
 'subsitut(?:e|es|ed|ing)',
 'substitut(?:e|es|ed|ing)',
 'un(?:-| )?hid(?:e|es|ed|ing)'

    #'chang(?:e|esed|ing)'
    #, 'creat(?:e|es|ed|ing)'
    #, 'delet(?:e|es|ed|ing)'
    #, 'instead'
    #, 'kill(?:ed|s|ing)?'
    # , 'provid(?:e|es|ed|ing)'
    #, 'introduc(?:e|es|ed|ing)'
] + core_refactor_terms + unnedded_terms

feedbak_terms = [ 'py(?:-| )?lint', 'lint', 'review comments(?:s)?', 'code review', 'cr', 'pep8'
                  ]
feedback_action = ['fix(?:ed|s|es|ing)?', 'fix(?:-| )?up(?:s)?', 'resolv(?:e|ed|es|ing)', 'correct(?:ed|s|es|ing)?']

perfective_header_action = [
    #'polish(?:ed|es|ing)?'
    #, 'clean(?:ing|s|ed)?(?:-| )?up(?:s)?'
     'clean(?:ing|s|ed)?(?:-| )?up(?:s)?'
    , 'cleaner'
    , 'deprecat(?:e|es|ed|ing)'
    , 'extract(?:ed|s|ing)?',
    're(?:-|)?organiz(?:e|es|ed|ing)', 're(?:-|)?structur(?:e|es|ed|ing)', 'tid(?:y|ying|ied) up'
    , 'improv(?:e|ed|es|ing|ement|ements)' , 're(?:-|)?organiz(?:e|es|ed|ing)', 're(?:-|)?structur(?:e|es|ed|ing)'
    , '(helper|utility|auxiliary) function(?:s)?'
    , '(?:move|moved|moves|moving) to'
    , 'separat(?:e|es|ed|ing)'
    , 'split(?:s|ing)?', '->'
    , build_sepereted_term(static_analyzers) + 'fix(es|ed)?'
    , 'fix(es|ed)?' + build_sepereted_term(static_analyzers)

    #, '(private|public|protected|static)'
]

# TODO - rewrited, move into/out???, deduplicate, remove legacy, redo, PR, feedback

# TODO - clean , style, prettier, "->", refine, "Removed commented code", "More startup improvements.", recode
# ""Remove another old function", "improved redis error message", utility functions, never used
# Checkstyle


# TODO - perfective, not refactor - ident, spacing, tabs, "tabs -> spaces", cosmetic, ""*** empty log message ***"
# examples ""DOC: remove mention of TimeSeries in docs"

# TODO - add "resolving review comments"
# TODO - lint, pylint
refactor_context = [ 'clean(ing)?(-| )?up(s)?'
    ,'call(?:s|ed|ing)?[\s\S]{1,50}instead'
    , 'collaps(?:e|es|ed|ing)', 'consolidat(e|es|ed|ing)'
    , 'decompos(?:e|es|ed|ing)'
    , 'drop(?:ed|s|ing)?( back)', 'encapsulat(e|es|ed|ing)'
    , 'gereneliz(?:e|es|ed|ing)'
                    # , 'inline'
                    # , 'no longer needed', 'not used', 'obsolete(d)?'
    , 'optimiz(?:e|es|ed|ing|ation|ations)'
    , 'pull(?:ed|s|ing)? (up|down)', 're(?:-)?(?:write|wrote)', 're(?:-| )?factor(?:ed|s|ing|ings)?'
    , 're(-)?implement(ed|s|ing)?'
    , 'renam(?:e|es|ed|ing|ings)', 'better nam(?:e|es|ing)','re(?:-)?organiz(e|es|ed|ing)', 're(?:-)?organization'
    , 're(?:-)?work(ed|s|ing|ings)?'
    , 'reorg' , 'simplif(y|es|ied|ying|ication)', 'suppress(es|ed|ing)? warning(?:s)?'
    , 'unif(?:y|ies|ied|ing)', 'uninline'
    , 'beef(?:ed|s|ing)? up', 'refactor(?:ing)?(?:s)?', 'code improvement(?:s)?'
    #, '(?:^|^[\s\S]{0,25}%s)(?:%s)%s[\s\S]{0,25}$' % (term_seperator, "|".join(perfective_header_action), term_seperator)
    , 'revis(?:e|es|ed|ing)'
    , 're(?:-)?construct(?:s|ed|ing)?'
    , 're(?:-)?(?:write|write|wrote|writing)'
    , 're(?:-)?cod(?:e|ed|es|ing)'
    , 'factor(?:ed|s|ing)? out'
    , 're(?:-| )?packag(?:e|es|ed|ing)'
    #, 'code review'
    #, 'collapse'
    #, "(?:(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)%s)" % (build_sepereted_term(feedback_action
    #                                                                                      , just_before=True)
    #                                                                 , term_seperator
    #                                                                 , term_seperator
    #                                                                 , term_seperator
    #                                                                 , "|".join(feedbak_terms)
    #                                                                 , term_seperator)
                    # ,'us(e|es|ed|ing)[\s\S]{1,50}(instead)'
                    # , '(instead)[\s\S]{1,50}us(e|es|ed|ing)'
                    ]
# https://refactoring.guru/refactoring/techniques

# TODO - change [\s\S] with . ?
removal = [ 'add(?:s|ed|ing)?[\s\S]{1,50}helper(?:s)?'
    ,  'us(?:e|es|ed|ing)[\s\S]{1,50}instead'
    #,  'us(?:e|es|ed|ing)[\s\S]{1,25}\(\)[\s\S]{1,25}instead'
    ,  'split(?:s|ing)?[\s\S]{1,50}into'
    ,  'break(?:s|ing)?[\s\S]{1,50}into'
    ,  'separat(?:e|e s|ed|ing)[\s\S]{1,50}into'
    #,  'replac(?:e|es|ed|ing)?[\s\S]{1,50}with'
    ,  'replac(?:e|es|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    , 'remov(?:e|es|ed|ing)[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    #, '(?:this|that|is)[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    ,  'kill(?:s|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    ,  'drop(?:s|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
    ,  'mov(?:e|es|ed|ing)?[\s\S]{1,50}(?:%s)' % "|".join(unnedded_terms)
            ]
adaptive_context = [
    '(?:un)?hid(?:e|es|den)', 'add(?:s|ed|ing)?', 'allow(?:s|ed|ing)?'
    , 'buil(?:t|ds|ing)', 'calibirat(?:e|es|ed|ing)'
    , 'configure'
    , 'creat(?:e|es|ing)' #   O created
    , 'deferr(?:ed|s|ing)?'
    , 'disabl(?:e|es|ed|ing)'
    , 'enhanc(?:e|es|ed|ing)', 'extend(?:s|ed|ing)?', 'form(?:ed|s|ing)?'
    , 'implement(?:ed|s|ing)?', 'import(?:s|ed|ing)?', 'introduc(?:e|es|ed|ing)'
    , 'port(?:s|ed|ing)?'
    , 'provid(?:e|es|ed|ing)'
    , 'report(?:s|ed|ing)?'
    , 'support(s|ed|ing)?'
    , 'updat(?:e|es|ed|ing)'
    , 'upgrad(?:e|es|ed|ing)'

    # , 'mov(e|es|ed|ing)'
    # , 'print(s|ed|ing)?'


]

In [ ]:
def build_core_refactor_regex():

    return '(%s)' % build_sepereted_term(core_refactor_terms)

def build_refactor_regex(use_conventional_commits=True):
    header_regex =  '(?:^|^[\s\S]{0,25}%s)(?:%s)%s' % (term_seperator
                                                       , "|".join(perfective_header_action)
                                                       , term_seperator)

    activity_regerx = "(?:(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)%s)" % (build_sepereted_term(modification_activity
                                                                                          , just_before=True)
                                                                     , term_seperator
                                                                     , term_seperator
                                                                     , term_seperator
                                                                     , "|".join(refactor_entities)
                                                                     , term_seperator)
    if use_conventional_commits:
        agg_re = "(%s)|(%s)|(%s)|(%s)" % (build_sepereted_term(refactor_context)
                          , activity_regerx
                          , header_regex
                          , build_cc_refactor_regex())
    else:
        agg_re = "(%s)|(%s)|(%s)" % (build_sepereted_term(refactor_context)
                          , activity_regerx
                          , header_regex)
    return agg_re



def build_refactor_goals_regex():
    goals_regerx = "(?:(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)%s)" % (build_sepereted_term(software_goals_modification
                                                                                       , just_before=True)
                                                                  , term_seperator
                                                                  , term_seperator
                                                                  , term_seperator
                                                                  , "|".join(software_goals)
                                                                  , term_seperator)
    return goals_regerx


def build_non_code_perfective_regex():

    non_perfective_entities = ['warning(?:s)?'
                               , 'format(?:ing)?'
                               , 'indentation(?:s)?'
                              ]
    # TODO - applied to perfective entities too here, which is a bug.
    modification_action = ['clean(?:-| )?up(?:s)?']
    non_perfective_context = [
                            'fix(?:es|ed|ing)?'
                            ,'(?:get|got|getting) rid'
                            , 'support(?:s|ed|ing)?'
                            ]
    modifiers = modification_activity + non_perfective_context
    activity_regerx = "((?:%s)(?:\s|%s[\s\S]{0,50}%s)(?:%s))" % (build_sepereted_term(modifiers, just_before=True)
                                                                                , term_seperator
                                                                                , term_seperator
                                                                                , "|".join(prefective_entities
                                                                                           + non_perfective_entities))
    doc_header_regex =  '(?:^|^[\s\S]{0,25}%s)(?:%s)[\s\S]{0,25}(?:%s)' % (term_seperator
                                                       , "|".join(perfective_header_action)
                                                       , build_sepereted_term(documentation_entities))


    no_prefective_action = "|".join([
        'convert(?:ed|s|ing)?(?:%s|%s[\s\S]{0,50}%s)support(?:s|ed|ing)?' % (
            term_seperator,term_seperator, term_seperator)
        , '(?:make|made|making|makes)(?:%s|%s[\s\S]{0,50}%s)work' % (term_seperator, term_seperator, term_seperator)
        , '(?:make|made|making|makes)(?:%s|%s[\s\S]{0,50}%s)sense' % (term_seperator, term_seperator, term_seperator)
        , 'improv(?:e|es|ed|ing) handling'
        , 'need(?:s|ing)?\srefactor(?:ing)?'
        , '(?:%s)(?:%s|%s[\s\S]{0,50}%s)(?:%s)' %(build_sepereted_term(non_perfective_entities,just_before=True)
                                                   ,term_seperator
                                                   , term_seperator
                                                   , term_seperator
                                                   , "|".join(modification_action)
                                                   )
        , doc_header_regex

    ])
    non_perfective_context = '(?:%s|%s)' % (no_prefective_action
                                         , activity_regerx)

    return non_perfective_context


def build_documentation_entities_context(positive_re):

    return '(?:%s)' % "|".join([
        # TODO - take care of documentation entities spereatly
         '(?:%s)[\s\S]{0,10}(?:%s)' % (build_sepereted_term(documentation_entities, just_before=True)
                                        ,positive_re)
    ])

def build_perfective_regex():
    non_code = build_sepereted_term (prefective_entities)

    perfective = "(%s)" %  non_code

    return perfective

**Classification Functions**

In [ ]:
def is_security(commit_text):

    return (len(re.findall(build_positive_regex(), commit_text))
            - len(re.findall(build_excluded_regex(), commit_text))
            - len(re.findall(build_not_positive_regex(), commit_text)))  > 0

In [ ]:
def is_core_bug(commit_text):
    text = commit_text.lower()

    cnt = len(re.findall(build_core_bug_regex(), text))

    return cnt > 0

In [ ]:
def is_fix(commit_text):

    text = commit_text.lower()
    #text = normalize(text)

    fix_num = len(re.findall(build_bug_fix_regex(), text))
    valid_num = len(re.findall(build_valid_find_regex(), text))
    negated_num = len(re.findall(build_negeted_bug_fix_regex(), text))
    # TODO  consider modals
    #negated_num = len(re.findall(build_non_positive_linguistic(build_bug_fix_regex()), text))
    return (fix_num - valid_num - negated_num) > 0
    #return (fix_num ) > 0 # max recall with current predictor

In [ ]:
def is_adaptive(text):

    x = (match(text, build_adaptive_regex())
            + match(text, build_adaptive_action_regex())
            - match(text, build_non_adaptive_context())
            - match(text, build_non_adaptive_linguistic()))
    
    if x > 0:
      return True
    else:
      return False

In [ ]:
def built_is_refactor(commit_text):
    removal_re = build_sepereted_term(removal)

    return (match(commit_text, build_refactor_regex())
            + match(commit_text, removal_re)
            + match(commit_text, build_refactor_goals_regex())
            - match(commit_text, build_non_code_perfective_regex())
            - match(commit_text
                    , build_documentation_entities_context(build_refactor_regex(use_conventional_commits=False)))
            - match(commit_text
                    , build_non_positive_linguistic(build_refactor_regex(use_conventional_commits=False)))
            - match(commit_text, build_non_positive_linguistic(build_sepereted_term(removal)))
            - match(commit_text, build_non_positive_linguistic(build_refactor_goals_regex()))
            ) > 0

This function returns a category based on the commit message. It is important to note the sequential manner that the messages are regex matched. Therefore, this order of funciton checking ensures that the latter functions has a broader vocabulary of keywords. This is a limitation of adding this category feature.

In [ ]:
def categorize_message_single(sample):

  if len(str(sample['message']).strip()) == 0:
    return 'other'

  if is_security(str(sample['message'])):
    return 'security'

  if is_fix(str(sample['message'])):
    return 'fix'

  if built_is_refactor(str(sample['message'])):
    return 'refactor'

  if is_core_bug(str(sample['message'])):
    return 'bug'

  if is_adaptive(str(sample['message'])):
    return 'other'

  return 'other'

These function adds the one hot labels, based on the previously identified categories.

In [ ]:
def cat_bug(cat):
  if str(cat['category']) == "bug":
    return 1
  else:
    return 0

def cat_other(cat):
  if str(cat['category']) == "other":
    return 1
  else:
    return 0

def cat_security(cat):
  if str(cat['category']) == "security":
    return 1
  else:
    return 0

def cat_fix(cat):
  if str(cat['category']) == "fix":
    return 1
  else:
    return 0

def cat_refactor(cat):
  if str(cat['category']) == "refactor":
    return 1
  else:
    return 0

In [ ]:
df['category'] = df.apply(categorize_message_single, axis=1)

In [ ]:
df.sample(10)

,commit,author,date,message,repo,category
1805462,bc4225320c87ac1d8e4bfd84719b227abf78c109,Mark Johnston <markj@FreeBSD.org>,Tue Oct 8 23:52:04 2019 +0000,Fix a bug in r353332 that snuck in with a last...,freebsd/freebsd-src,fix
415020,2e0d6d49bf6c95185229ac19f8b1c43f1bca31fd,bjorn3 <bjorn3@users.noreply.github.com>,Sat Aug 11 13:59:08 2018 +0200,Deduplicate function name generation,rust-lang/rust,other
1977842,a30d7c60f602d4c7c983e3f91aea34518e094567,Jake Burkholder <jake@FreeBSD.org>,Sat Apr 6 08:13:52 2002 +0000,Use CTASSERT rather than a runtime check to de...,freebsd/freebsd-src,other
3861214,0c998f41594caadfb00d22e512c3e06f247e24de,abarth@chromium.org <abarth@chromium.org@bbb92...,Thu May 30 18:32:28 2013 +0000,Add Adam Klein as a Core OWNER\n \n He w...,chromium/chromium,other
2840851,e0a70217107e6f9844628120412cb27bb4cea194,Oleg Nesterov <oleg@redhat.com>,Fri Nov 5 16:53:42 2010 +0100,posix-cpu-timers: workaround to suppress the p...,torvalds/linux,fix
3747827,76b735208764148d18bac5e1c91ec20cd464c01b,garykac <garykac@chromium.org>,Tue Nov 18 09:28:10 2014 -0800,[Chromoting] Add auth dialog to window shape\n...,chromium/chromium,security
3885810,ebe123f9aeb87467c0cdf8145d8194811f1decc0,nfullagar@chromium.org <nfullagar@chromium.org...,Wed Feb 13 01:10:22 2013 +0000,Hush compiler warnings when compiling c code.\...,chromium/chromium,fix
1847141,4a0589d1ba7b8738639f5d9f6ef585531c1990f4,Konstantin Belousov <kib@FreeBSD.org>,Thu Aug 20 13:37:08 2015 +0000,Typo.,freebsd/freebsd-src,other
3021469,bed8bdfddd851657cf9e5fd16bb44abb02ae7f42,Eric Sesterhenn <snakebyte@gmx.de>,Mon Oct 23 22:17:21 2006 +0200,IB: kmemdup() cleanup\n \n Replace open ...,torvalds/linux,other
3477480,df16d7801416be35e6cd8f0dd473ad981baa3fc7,Charlie Harrison <csharrison@chromium.org>,Tue May 15 22:42:17 2018 +0000,Remove SubresourceFilter.SafeBrowsing.CheckDis...,chromium/chromium,fix


In [ ]:
df.to_csv(f'/content/github_categories.csv', index = False, encoding = 'utf-8') 

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import shutil
# shutil.copy("/content/github_categories.csv", "/content/gdrive/MyDrive/") 

'/content/gdrive/MyDrive/github_categories.csv'

Adding One Hot Labels

In [ ]:
newdf = df

In [ ]:
newdf['other'] = newdf.apply(cat_other, axis=1)

In [ ]:
newdf['bug'] = newdf.apply(cat_bug, axis=1)
newdf['fix'] = newdf.apply(cat_fix, axis=1)
newdf['refactor'] = newdf.apply(cat_refactor, axis=1)
newdf['security'] = newdf.apply(cat_security, axis=1)

In [ ]:
newdf.sample(10)

,id,message,other,bug,fix,refactor,security
3804104,384b92661c8bfa0c80b92cfa4d1118cd9de0dc6d,"Revert 254791 ""Disable HostDriven_SyncTest.tes...",0,0,1,0,0
4073714,e360fac83f6c27336fc2411ac19ba4a75f4b0683,Since the host content settings preferences ca...,0,0,1,0,0
2386411,c499a64f349d063d8cdb40c0b96e84c35bbc414c,net: dsa: mv88e6xxx: move VTU Data accessors\n...,0,0,0,1,0
2582938,f74954f01ec9bb2004bcc24f247d1f26f1063ad2,x86: Unwind-annotate thunk_32.S\n \n Sig...,1,0,0,0,0
3995118,74b7d742acf4dbde32d4486d747be82ea9cda039,Update .DEPS.git\n \n git-svn-id: svn://...,1,0,0,0,0
2784135,a7e3bd669eb407b4e700a023e502bfc80d814b04,staging:iio:light:tsl2563 both intensity chann...,0,0,1,0,0
3728489,47904be75e5fdea64ba0b4c49472b722aabf6a43,Roll src/third_party/skia b421650:7ab4277\n ...,1,0,0,0,0
2239326,18c0778a650b21f8729576b7bc382447d2027d4d,NFSv4: Handle early exit in layoutget by retur...,0,0,1,0,0
2967420,94ee1cf5a88e12f5cbf8c0c78a6c18d3e043241e,edac: add e752x parameter for sysbus_parity se...,0,0,1,0,0
1850557,17d52d23512fd5913fe6628609a57744dfdb6ba3,Improve smb(4) man page.\n \n Differenti...,1,0,0,0,0


In [ ]:
newdf = newdf.drop(['author', 'date', 'repo', 'category'], axis = 1)

In [ ]:
newdf.rename(columns = {'commit':'id'}, inplace = True)

In [ ]:
newdf.to_csv(f'/content/github_training.csv', index = False, encoding = 'utf-8') 

In [ ]:
# shutil.copy("/content/github_training.csv", "/content/gdrive/MyDrive/") 

'/content/gdrive/MyDrive/github_training.csv'